---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [82]:
import pandas as pd
import re as regex
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.iloc[300:350]

300    .Since January 1994, she feels that hse has be...
301    S/p colectomy in Dec 1992.Problems ANXIETY : D...
302    Hx of 3 instances that pt believes were concus...
303    7 first psych hospital approx age 20, most rec...
304    past suicidal gestures by overdose (trazodone/...
305    . The patient was hospitalized in Feb 2000 at ...
306                   May, 2004 Hx of Brain Injury: No\n
307    invasive ductal carcinoma of the left breast(s...
308    dCelexa: most helpful, came off half-way throu...
309    Pt put on a 504 at the beginning of 10 th grad...
310    eZoloft : dizziness and tiredness (only 1 dose...
311    - Prozac 20 mg daily:  February, 1995: self-di...
312    .On review of systems, she reports that her sl...
313    nLoss of father to cardiac event in Decemeber ...
314                           a Saddle PE January 2007\n
315    rBrookhaven outpatient program in Jun 1976- lo...
316    2 cats-3 family house-planning to move in May ...
317    . Psychosocial: lives w/

In [129]:
def date_sorter():
    
    answer_df = pd.DataFrame()
    
    #mm/dd/yy
    date_format1 = '(?P<month>\d{1,2})[-/](?P<day>\d{1,2})[-/](?P<year>\d{4})'
    df1 = df.str.extractall(date_format1)
    
    #two-digit-years
    date_format2 = '(?P<month>\d{1,2})[-/](?P<day>\d{1,2})[-/](?P<year>\d{2})'
    df2 = df.str.extractall(date_format2)
    df2['year'] = df2['year'].apply(lambda x: '19'+x)
    
    #day is missing
    date_format3 = '(?P<month>\d{1,2})[-/](?P<year>\d{4})'
    df3 = df.str.extractall(date_format3)
    df3['day'] = 1
    #to rearrange the columns
    df3 = df3[['month', 'day', 'year']]
  
    #only year
    date_format4 = '(?P<year>[12]\d{3})'
    df4 = df.str.extractall(date_format4)
    df4['month'] = 1
    df4['day'] = 1
    
    #months with words
    date_format5 = '(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w*)[\s\.-]+(?P<day>\d{1,2})\D{1,2}(?P<year>\d{4})'
    df5 = df.str.extractall(date_format5)
    #create dictionary for months
    months = {'Jan': 1, 'January':1,
                  'Feb': 2, 'February': 2,
                  'March': 3, 'Mar':3,
                  'Apr': 4, 'April':4,
                  'May': 5,
                  'June': 6, 'Jun': 6,
                  'July': 7, 'Jul': 7,
                  'Aug': 8, 'August': 8,
                  'September': 9, 'Sep': 9,
                  'October': 10, 'Oct': 10,
                  'Nov': 11, 'November': 11,
                  'Dec': 12, 'December': 12}
    #df5.replace(months, inplace=True)
    
    date_format6 = '(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))\D{1,}(?P<year>\d{4})'
    df6 = df.str.extractall(date_format6)
    df6['day'] = 1
    #to rearrange the columns
    df6 = df6[['month', 'day', 'year']]
    
    date_format7 = '(?P<day>\d{1,2})[\s\.-]*(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))\D{1,}(?P<year>\d{4})'
    df7 = df.str.extractall(date_format7)
    #to rearrange the columns
    df7 = df7[['month', 'day', 'year']]
    
    frames = [df1, df2, df3, df4, df5, df6, df7]
    answer_df = pd.concat(frames)
    
    #removing duplicates by cleaning some data
    answer_df.replace(months, inplace=True)
    answer_df['day'] = answer_df['day'].apply(lambda x: str(x))
    answer_df['month'] = answer_df['month'].apply(lambda x: str(x))
    answer_df['year'] = answer_df['year'].apply(lambda x: str(x))
    answer_df['day'] = answer_df['day'].apply(lambda x: x[1:] if(x[0]=='0') else x)
    answer_df['month'] = answer_df['month'].apply(lambda x: x[1:] if(x[0]=='0') else x)
    answer_df = answer_df.drop_duplicates().reset_index().drop('level_0',axis=1)
    answer_df['full'] = answer_df['month'] +'/'+ answer_df['day'] + '/' + answer_df['year']
    #answer_df['full'] = pd.to_datetime(answer_df.full)
    answer_df.drop(176,inplace=True, axis=0)
    answer_df = answer_df.sort_values(by='year')
    return pd.Series(answer_df.index)

date_sorter()

0       84
1       80
2       39
3       42
4      103
5       89
6      121
7       70
8       49
9       63
10      55
11     111
12     120
13     108
14      94
15      68
16      40
17     118
18      83
19      59
20     144
21     135
22     109
23     132
24     128
25      98
26      78
27     110
28      34
29      53
      ... 
499    224
500    476
501    348
502    451
503    290
504      5
505    336
506    434
507    156
508    367
509    317
510    326
511    262
512    433
513    215
514    200
515    232
516    270
517    377
518    311
519    488
520    409
521    418
522    521
523    383
524    258
525    497
526    478
527    307
528    244
Length: 529, dtype: int64